In [9]:
def extractGribData(fileurl,latitude1, longitude1):
    "The function extract all the data for the given longitudes (-0.5->359.5) and latitudes(-90.5->90.5). Output is given as a dataframe"
    data = pd.DataFrame(columns=['datetime_of_origin', 'datetime_of_forecast', 'latitude', 'longitude', 'parameter_type', 'parameter_value', 'unit_value', 'unit_description']) #Generating data frame for input
    grbs = pygrib.open(fileurl) #Open grib file as iterator
    i=0 #Index counter
    
    #Loop through the messages and then pick the longitude and latitude
    for msg in grbs:
            data.at[i,'parameter_type']=msg.name #Parameter type
            data.at[i, 'datetime_of_origin']=msg.analDate #Datetime for when forecast was generated
            data.at[i, 'datetime_of_forecast']=msg.validDate #Datetime for when forecast was valid
            data.at[i, 'unit_description']=msg.typeOfLevel #Unit description
            data.at[i, 'unit_value']=msg.level #Unit value
            data.at[i, 'parameter_value']=float(msg.data(lat1=latitude1, lat2=latitude1, lon1=longitude1, lon2=longitude1)[0][0][0]) #Parameter value        
            data.at[i, 'latitude']=msg.data(lat1=latitude1, lat2=latitude1, lon1=longitude1, lon2=longitude1)[1][0][0] #Latitude   
            data.at[i, 'longitude']=msg.data(lat1=latitude1, lat2=latitude1, lon1=longitude1, lon2=longitude1)[2][0][0] #Longitude   
            i=i+1
    return data

In [10]:
#Purpose: This script is made to extract data from a grib2 file for only a specific latitude and longitude, and then insert it into a pickle. 
#Date: 13.10.2016 
#Author: ST

#Comment on grib files
#Files have different length. Files that forecast ahead includes average for some values. For example 0-6 hour avg.
#Files include have values which can be for example temperature, windspeed and humidity. This is at different levels.
#Then we have different keys for each value. Inside here we have for example the grid with latitude and longitude.

#Comment on function 
#The function only take longitude and latitude but could be generalized to take a range of longitude and latitudes.

#Import relevant libraries
import pygrib
import numpy as np
from datetime import datetime
import pandas as pd
import time

start_time=time.time()

#Generate a data frame to store values from files. Need to be on same form as output of the "extractGribData" function
df = pd.DataFrame(columns=['datetime_of_origin', 'datetime_of_forecast', 'latitude', 'longitude', 'parameter_type', 'parameter_value', 'unit_value', 'unit_description']) 

#Defining path and file variables
output_path="c/Users/stto/OneDrive - Powel AS/Prosjekter/PV Forecast, Chile/GFS/data.pickle"
input_path="c/Users/stto/OneDrive - Powel AS/Prosjekter/PV Forecast, Chile/GFS/"
filenames=["gfs_4_20161001_0000_000.grb2", "gfs_4_20161001_0000_003.grb2", "gfs_4_20161001_0000_006.grb2"]

#Loop over input files and append date to the data frame
for filename in filenames:
    fileurl=input_path+filename
    print("Reading data from " + filename)
    df=df.append(extractGribData(filename, 20, 25), ignore_index=True) #Use fileurl if grib file is not in the same folder as the python script. Could be problematic with Docker?

#Quality checking
print(df.head(5))
print(df.tail(5))

#Pickle the data frame
print("Transforming data frame to pickle")
df.to_pickle("data.pickle")

print("Total time to process was: ")
print(time.time()-start_time)

Reading data from gfs_4_20161001_0000_000.grb2


/opt/conda/lib/python2.7/site-packages/numpy/ma/core.py:4144: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


Reading data from gfs_4_20161001_0000_003.grb2
Reading data from gfs_4_20161001_0000_006.grb2
    datetime_of_origin datetime_of_forecast latitude longitude  \
0  2016-10-01 00:00:00  2016-10-01 00:00:00       20        25   
1  2016-10-01 00:00:00  2016-10-01 00:00:00       20        25   
2  2016-10-01 00:00:00  2016-10-01 00:00:00       20        25   
3  2016-10-01 00:00:00  2016-10-01 00:00:00       20        25   
4  2016-10-01 00:00:00  2016-10-01 00:00:00       20        25   

        parameter_type parameter_value unit_value unit_description  
0  U component of wind            -8.1          0          unknown  
1  V component of wind              -9          0          unknown  
2     Ventilation Rate            3000          0          unknown  
3    Wind speed (gust)            12.7          0          surface  
4  Geopotential Height         47664.3          1    isobaricInhPa  
       datetime_of_origin datetime_of_forecast latitude longitude  \
1177  2016-10-01 00:00:00 

In [13]:
df.dtypes

datetime_of_origin      object
datetime_of_forecast    object
latitude                object
longitude               object
parameter_type          object
parameter_value         object
unit_value              object
unit_description        object
dtype: object